In [5]:
# artifacts.py
from artifacts import *
from data import load_data, batchify

# setup
config = {
"embedding_dimension": 200,
"ff_dimension": 200,
"n_attention_heads": 2,
"n_encoder_layers": 0,
"n_decoder_layers": 2,
"dataset": "PennTreebank",
"segmentation": "Word",
"max_seq_len": 35,
"batch_size": 20,
"eval_batch_size": 10,
"dropout": 0.2,
"n_epochs": 3,
"learning_rate": 0.5,
"loss_criterion": "CrossEntropyLoss"
}

# extract config
batch_size, eval_batch_size = extract_config(config, "batch_size", "eval_batch_size")

# configure device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load training data
train_data, val_data, test_data, vocab = load_data(config)
ntokens = len(vocab.stoi)

# batch data
train_data_batches = batchify(train_data, batch_size, device)
val_data_batches = batchify(val_data, eval_batch_size, device)
test_data_batches = batchify(test_data, eval_batch_size, device)


# testing
artifacts = initalize_artifacts(config, train_data_batches, val_data_batches)
update_artifact_loss(artifacts, 'training', 'CrossEntropyLoss', 0, 1, 0.5)
update_artifact_loss(artifacts, 'training', 'CrossEntropyLoss', 0, 2, 3)
# artifacts['training']['CrossEntropyLoss'].reshape(-1)
visualize_artifacts(artifacts)

ModuleNotFoundError: No module named 'torch'